## **Trabalho 1 (Classificação) - Introdução ao Aprendizado de Máquina (EEL891)**
> Nome: Danilo Davi Gomes Fróes
>
> DRE: 124026825

Este é o trabalho 1 da matéria Int. ao Aprendiz. de Máquina, ministrada pelo professor Heraldo Almeida na UFRJ. O tema do trabalho é `Classificação: sistema de apoio à decisão p/ aprovação de crédito`, no qual será construído um classificador para apoio à decisões.

O trabalho foi desenvolvido usando conhecimentos aprendidos durante as aulas e também do meu projeto de iniciação científica no Laboratório de Simulação e Otimização de Sistemas Produtivos (LASOS) do Instituto Nacional de Tecnologia (INT). O projeto é para o desenvolvimento de uma biblioteca em python de machine learning para otimizar a produção de múltiplas pipelines e gerar insights úteis de retorno para o usuário sem a necessidade de construção de grandes códigos.
Deixo aqui um resumo recente do trabalho desenvolvido até então, que futuramente será lançada publicamente para uso da comunidade do python.

[Resumo - LasosLibrary](https://docs.google.com/document/d/1T4g_4vvwjcM2SF-lBHUC7RIizCHBxomc/edit?usp=sharing&ouid=107178103419237221884&rtpof=true&sd=true)

### **Análise Exploratória de Dados (EDA - Exploratory Data Analysis)**
Uma parte essencial para treinar modelos de machine learning é entender o que são os dados que estamos utilizando, dessa forma, fiz uma abordagem utilizando funções da biblioteca pandas e o dicionário de dados disponibilizado para obter informações essenciais para o desenvolvimento do projeto.

- Nas células abaixo, realizo a obtenção dos dados em CSV como `DataFrames` do pandas e utilizo as funções `info`, `describe` e `value_counts` para uma análise mais aprofundada de cada feature e do balanceamento da target.

In [31]:
import pandas as pd

df_treinamento = pd.read_csv('conjunto_de_treinamento.csv')

print(f'Info: {df_treinamento.info()}')

print(f'Descrição: {df_treinamento.describe()}')

print(f'Targets: {df_treinamento['inadimplente'].value_counts()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 42 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id_solicitante                    20000 non-null  int64  
 1   produto_solicitado                20000 non-null  int64  
 2   dia_vencimento                    20000 non-null  int64  
 3   forma_envio_solicitacao           20000 non-null  object 
 4   tipo_endereco                     20000 non-null  int64  
 5   sexo                              20000 non-null  object 
 6   idade                             20000 non-null  int64  
 7   estado_civil                      20000 non-null  int64  
 8   qtde_dependentes                  20000 non-null  int64  
 9   grau_instrucao                    20000 non-null  int64  
 10  nacionalidade                     20000 non-null  int64  
 11  estado_onde_nasceu                20000 non-null  object 
 12  esta

In [32]:
df_predicao = pd.read_csv('conjunto_de_teste.csv')

print(f'Info: {df_predicao.info()}')

print(f'Descrição: {df_predicao.describe()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id_solicitante                    5000 non-null   int64  
 1   produto_solicitado                5000 non-null   int64  
 2   dia_vencimento                    5000 non-null   int64  
 3   forma_envio_solicitacao           5000 non-null   object 
 4   tipo_endereco                     5000 non-null   int64  
 5   sexo                              5000 non-null   object 
 6   idade                             5000 non-null   int64  
 7   estado_civil                      5000 non-null   int64  
 8   qtde_dependentes                  5000 non-null   int64  
 9   grau_instrucao                    5000 non-null   int64  
 10  nacionalidade                     5000 non-null   int64  
 11  estado_onde_nasceu                5000 non-null   object 
 12  estado

### **Tratamento dos Dados**
Aqui é onde vai ser feito o tratamento dos dados após a análise geral. A ideia é deixar apenas dados e criar dados que possuam alto valor de predição para o modelo, evitando vieses e desvios desnecessárias.

#### **Remoção de colunas**
Algumas colunas foram vistas como problemáticas e serão removidas, elas são:
- `id_solicitante`: Identificador único, não é necessário para o modelo.
- `grau_instrucao`: Veio preenchida apenas com zeros.
- `possui_telefone_celular`: Veio preenchida apenas com `'N'`.
- `qtde_contas_bancarias_especiais`: Idêntica a outra coluna, `'qtde_contas_bancarias'`.

#### **Substituição de valores nulos ocultos**
Foi observado pelo dicionário de dados, que existem valores nulos que estão ocultos no dataframe, ou seja, não seriam enxergues como nulos no pré-processamento. Valores como `"XX"`, espaços em branco `" "`, `NULL`, dentre outros serão substituídos por valores nulos NaN.

#### **Engenharia de Features**
Essa parte foi essencial para melhor o potencial de predição dos modelos. Aqui foi estudado um pouco de todas as features que tínhamos e como poderíamos unir algumas para criar outras com maior valor de predição.
- `renda_total`: Essa feature foi criada realizando a soma da `renda_mensal_regular` e da `renda_extra`.
- `patrimonio_por_renda`: Essa feature foi criada com intuito de trazer a relação de patrimônio e renda para a predição, utilizando o `valor_patrimonio_pessoal` dividido pela `renda_total`.

#### **Tratamento de valores extremos**
Foi constatado em algumas features dados que fugiam muito da curva padrão. Dessa forma, para evitar overfitting, realizei o tratamento desses dados.
- `qtde_dependentes`: Esse dado foi limitado a 10 depedentes, pois tinham algumas situação onde os valores extrapolavam a realidade.
- `idade`: Foi considerado que menores de 17 anos não pudessem realizar

In [33]:
import numpy as np

# Remover colunas que não são necessárias para o modelo
colunas_a_remover = [
    'id_solicitante',
    'grau_instrucao',
    'possui_telefone_celular',
    'qtde_contas_bancarias_especiais'
]
df_treinamento = df_treinamento.drop(columns=colunas_a_remover, axis=1)

# Substituir valores inválidos por NaN
df_treinamento = df_treinamento.replace([' ', 'N/A', '', '?', 'XX', 'NULL'], np.nan)

# Engenharia de Features
df_treinamento['renda_total'] = df_treinamento['renda_mensal_regular'] + df_treinamento['renda_extra']
df_treinamento['patrimonio_por_renda'] = df_treinamento['valor_patrimonio_pessoal'] / (df_treinamento['renda_total'] + 1)
df_treinamento['renda_total_log'] = np.log1p(df_treinamento['renda_total'])
df_treinamento['nasceu_onde_reside'] = (df_treinamento['estado_onde_nasceu'] == df_treinamento['estado_onde_reside']).astype(int)
df_treinamento['reside_onde_trabalha'] = (df_treinamento['estado_onde_reside'] == df_treinamento['estado_onde_trabalha']).astype(int)
colunas_cartoes = ['possui_cartao_visa', 'possui_cartao_mastercard', 'possui_cartao_diners', 'possui_cartao_amex', 'possui_outros_cartoes']
df_treinamento['qtde_total_de_cartoes'] = df_treinamento[colunas_cartoes].sum(axis=1)

bins = [17, 25, 40, 60, 110]
labels = ['Jovem', 'Adulto', 'MeiaIdade', 'Senior']
df_treinamento['faixa_etaria'] = pd.cut(df_treinamento['idade'], bins=bins, labels=labels, right=False)

# Tratamento de valores extremos
df_treinamento.loc[df_treinamento['qtde_dependentes'] > 10, 'qtde_dependentes'] = 10
df_treinamento.loc[df_treinamento['idade'] < 17, 'idade'] = df_treinamento['idade'].median()
# df_treinamento.loc[df_treinamento['meses_no_trabalho'] < 1, 'meses_no_trabalho'] = df_treinamento['meses_no_trabalho'].median()

colunas_redundantes = [
    'renda_mensal_regular', 
    'renda_extra',
    'renda_total',
    'valor_patrimonio_pessoal'
]

df_treinamento = df_treinamento.drop(columns=colunas_redundantes, errors='ignore')

# Tratamento de valores nulos
cols_na_subs = ['profissao_companheiro', 'grau_instrucao_companheiro', 'codigo_area_telefone_residencial', 'codigo_area_telefone_trabalho', 'profissao', 'ocupacao']
for col in cols_na_subs:
    df_treinamento[col] = df_treinamento[col].fillna('-1')

cols_cat_subs = ['estado_onde_trabalha', 'estado_onde_nasceu', 'tipo_residencia'] 
for col in cols_cat_subs:
    moda = df_treinamento[col].mode()[0]
    df_treinamento[col] = df_treinamento[col].fillna(moda)

cols_num_subs = ['meses_na_residencia'] 
for col in cols_num_subs:
    mediana = df_treinamento[col].median()
    df_treinamento[col] = df_treinamento[col].fillna(mediana)

df_treinamento['sexo'] = df_treinamento['sexo'].fillna('N')

print(df_treinamento.info())
print(df_treinamento.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   produto_solicitado                20000 non-null  int64   
 1   dia_vencimento                    20000 non-null  int64   
 2   forma_envio_solicitacao           20000 non-null  object  
 3   tipo_endereco                     20000 non-null  int64   
 4   sexo                              20000 non-null  object  
 5   idade                             20000 non-null  int64   
 6   estado_civil                      20000 non-null  int64   
 7   qtde_dependentes                  20000 non-null  int64   
 8   nacionalidade                     20000 non-null  int64   
 9   estado_onde_nasceu                20000 non-null  object  
 10  estado_onde_reside                20000 non-null  object  
 11  possui_telefone_residencial       20000 non-null  obje

In [34]:
# Remover colunas que não são necessárias para o modelo
colunas_a_remover = [
    'id_solicitante',
    'grau_instrucao',
    'possui_telefone_celular',
    'qtde_contas_bancarias_especiais'
]
df_predicao = df_predicao.drop(columns=colunas_a_remover, axis=1)

# Substituir valores inválidos por NaN
df_predicao = df_predicao.replace([' ', 'N/A', '', '?', 'XX', 'NULL'], np.nan)

# Engenharia de Features
df_predicao['renda_total'] = df_predicao['renda_mensal_regular'] + df_predicao['renda_extra']
df_predicao['patrimonio_por_renda'] = df_predicao['valor_patrimonio_pessoal'] / (df_predicao['renda_total'] + 1)
df_predicao['renda_total_log'] = np.log1p(df_predicao['renda_total'])
df_predicao['nasceu_onde_reside'] = (df_predicao['estado_onde_nasceu'] == df_predicao['estado_onde_reside']).astype(int)
df_predicao['reside_onde_trabalha'] = (df_predicao['estado_onde_reside'] == df_predicao['estado_onde_trabalha']).astype(int)
colunas_cartoes = ['possui_cartao_visa', 'possui_cartao_mastercard', 'possui_cartao_diners', 'possui_cartao_amex', 'possui_outros_cartoes']
df_predicao['qtde_total_de_cartoes'] = df_predicao[colunas_cartoes].sum(axis=1)

bins = [17, 25, 40, 60, 110]
labels = ['Jovem', 'Adulto', 'MeiaIdade', 'Senior']
df_treinamento['faixa_etaria'] = pd.cut(df_treinamento['idade'], bins=bins, labels=labels, right=False)

# Tratamento de valores extremos
df_predicao.loc[df_predicao['qtde_dependentes'] > 10, 'qtde_dependentes'] = 10
df_predicao.loc[df_predicao['idade'] < 17, 'idade'] = df_predicao['idade'].median()
# df_predicao.loc[df_predicao['meses_no_trabalho'] < 1, 'meses_no_trabalho'] = df_predicao['meses_no_trabalho'].median()

colunas_redundantes = [
    'renda_mensal_regular', 
    'renda_extra',
    'renda_total',
    'valor_patrimonio_pessoal'
]

df_predicao = df_predicao.drop(columns=colunas_redundantes, errors='ignore')

# Tratamento de valores nulos
cols_na_subs = ['profissao_companheiro', 'grau_instrucao_companheiro', 'codigo_area_telefone_residencial', 'codigo_area_telefone_trabalho', 'profissao', 'ocupacao']
for col in cols_na_subs:
    df_predicao[col] = df_predicao[col].fillna('-1')

cols_cat_subs = ['estado_onde_trabalha', 'estado_onde_nasceu', 'tipo_residencia'] 
for col in cols_cat_subs:
    moda = df_predicao[col].mode()[0]
    df_predicao[col] = df_predicao[col].fillna(moda)

cols_num_subs = ['meses_na_residencia'] 
for col in cols_num_subs:
    mediana = df_predicao[col].median()
    df_predicao[col] = df_predicao[col].fillna(mediana)

df_predicao['sexo'] = df_predicao['sexo'].fillna('N')

df_predicao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 39 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   produto_solicitado                5000 non-null   int64  
 1   dia_vencimento                    5000 non-null   int64  
 2   forma_envio_solicitacao           5000 non-null   object 
 3   tipo_endereco                     5000 non-null   int64  
 4   sexo                              5000 non-null   object 
 5   idade                             5000 non-null   int64  
 6   estado_civil                      5000 non-null   int64  
 7   qtde_dependentes                  5000 non-null   int64  
 8   nacionalidade                     5000 non-null   int64  
 9   estado_onde_nasceu                5000 non-null   object 
 10  estado_onde_reside                5000 non-null   object 
 11  possui_telefone_residencial       5000 non-null   object 
 12  codigo

## Pré-processamento dos dados

In [37]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

X = df_treinamento.drop(columns=['inadimplente'])
y = df_treinamento['inadimplente']

colunas_categoricas = [
    # Códigos numéricos que são categorias
    'produto_solicitado', 
    'dia_vencimento', 
    'tipo_endereco',
    'estado_civil', 
    'nacionalidade', 
    'tipo_residencia', 
    'local_onde_reside',      # <--- AJUSTADO
    'local_onde_trabalha',    # <--- AJUSTADO

    # Colunas de texto (object)
    'forma_envio_solicitacao', 
    'sexo', 
    'estado_onde_nasceu',
    'estado_onde_reside', 
    'possui_telefone_residencial', 
    'codigo_area_telefone_residencial',
    'possui_telefone_trabalho', 
    'estado_onde_trabalha',
    'codigo_area_telefone_trabalho', 
    'vinculo_formal_com_empresa',
    'profissao', 
    'ocupacao',
    'profissao_companheiro', 
    'grau_instrucao_companheiro',
    'faixa_etaria',

    # Colunas de Flags Binárias (0/1)
    'possui_email',
    'possui_cartao_visa',
    'possui_cartao_mastercard',
    'possui_cartao_diners',
    'possui_cartao_amex',
    'possui_outros_cartoes',
    'possui_carro',
    'nasceu_onde_reside',     # <--- AJUSTADO
    'reside_onde_trabalha'    # <--- AJUSTADO
]

for col in colunas_categoricas:
    # Verifica se a coluna realmente existe no DataFrame antes de tentar converter
    if col in X.columns:
        X[col] = X[col].astype(str)

# As colunas numéricas são todas as que NÃO estão na lista de categóricas.
colunas_numericas = [col for col in X.columns if col not in colunas_categoricas]

print(f"\n{len(colunas_categoricas)} Colunas Categóricas:\n{sorted(colunas_categoricas)}")
print(f"\n{len(colunas_numericas)} Colunas Numéricas:\n{sorted(colunas_numericas)}")

scalers_a_testar = [
    ('StandardScaler', StandardScaler()),
    ('MinMaxScaler', MinMaxScaler()),
    # ('RobustScaler', RobustScaler()),
    ('QuantileTransformer', QuantileTransformer(output_distribution="normal")),
    ('PowerTransformer', PowerTransformer(method="yeo-johnson"))
]

encoders_a_testar = [
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore', drop='first')),
    # ('OrdinalEncoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
]

modelos_a_testar = {
    'LightGBM': {
        'model': LGBMClassifier(random_state=42),
        'params': {
            'classifier__n_estimators': [100, 200],
            'classifier__learning_rate': [0.05, 0.1],
            'classifier__max_depth': [5, 7]
        }
    },
    'CatBoost': {
        # verbose=0 para não poluir a saída do loop
        'model': CatBoostClassifier(random_state=42, verbose=0),
        'params': {
            'classifier__iterations': [100, 200],
            'classifier__learning_rate': [0.05, 0.1],
            'classifier__depth': [4, 6]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
        'params': {
            'classifier__n_estimators': [100, 200],
            'classifier__max_depth': [3, 5],
            'classifier__learning_rate': [0.1, 0.2]
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(random_state=42, max_iter=2000),
        'params': {
            'classifier__C': [0.1, 1.0, 10],
            'classifier__solver': ['liblinear', 'saga']
        }
    },
    # 'GradientBoosting': {
    #     'model': GradientBoostingClassifier(random_state=42),
    #     'params': {
    #         'classifier__n_estimators': [100, 200],
    #         'classifier__learning_rate': [0.05, 0.1],
    #         'classifier__max_depth': [3, 5]
    #     }
    # },
    # 'KNeighbors': {
    #     'model': KNeighborsClassifier(),
    #     'params': {
    #         'classifier__n_neighbors': [3, 5, 7],
    #         'classifier__weights': ['uniform', 'distance']
    #     }
    # },
    # 'DecisionTree': {
    #     'model': DecisionTreeClassifier(random_state=42),
    #     'params': {
    #         'classifier__max_depth': [5, 10, None],
    #         'classifier__min_samples_leaf': [2, 4],
    #         'classifier__criterion': ['gini', 'entropy']
    #     }
    # },
    # 'RandomForest': {
    #     'model': RandomForestClassifier(random_state=42),
    #     'params': {
    #         'classifier__n_estimators': [100, 200],
    #         'classifier__max_depth': [5, 10],
    #         'classifier__min_samples_leaf': [2, 4]
    #     }
    # }
}

# modelos_a_testar = [
#     ('LogisticRegression', LogisticRegression(random_state=42, max_iter=1000)),
#     ('RandomForest', RandomForestClassifier(random_state=42)),
#     ('XGBoost', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')),
#     ('DecisionTree', DecisionTreeClassifier(random_state=42)),
#     ('KNeighborsClassifier', KNeighborsClassifier()),
#     ('GradientBoostingClassifier', GradientBoostingClassifier(random_state=42)),
#     ('CatBoostClassifier', CatBoostClassifier(random_state=42, verbose=0)),
#     ('LightGBMClassifier', LGBMClassifier(random_state=42)),
#     ('ExtraTreesClassifier', ExtraTreesClassifier(random_state=42)),
#     ('SVM', SVC(random_state=42, probability=True))
# ]


32 Colunas Categóricas:
['codigo_area_telefone_residencial', 'codigo_area_telefone_trabalho', 'dia_vencimento', 'estado_civil', 'estado_onde_nasceu', 'estado_onde_reside', 'estado_onde_trabalha', 'faixa_etaria', 'forma_envio_solicitacao', 'grau_instrucao_companheiro', 'local_onde_reside', 'local_onde_trabalha', 'nacionalidade', 'nasceu_onde_reside', 'ocupacao', 'possui_carro', 'possui_cartao_amex', 'possui_cartao_diners', 'possui_cartao_mastercard', 'possui_cartao_visa', 'possui_email', 'possui_outros_cartoes', 'possui_telefone_residencial', 'possui_telefone_trabalho', 'produto_solicitado', 'profissao', 'profissao_companheiro', 'reside_onde_trabalha', 'sexo', 'tipo_endereco', 'tipo_residencia', 'vinculo_formal_com_empresa']

8 Colunas Numéricas:
['idade', 'meses_na_residencia', 'meses_no_trabalho', 'patrimonio_por_renda', 'qtde_contas_bancarias', 'qtde_dependentes', 'qtde_total_de_cartoes', 'renda_total_log']


In [38]:
from sklearn.model_selection import StratifiedKFold
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
resultados_finais = []
output_path = Path.cwd() / 'output_matrizes_otimizadas'
output_path.mkdir(exist_ok=True)

print("🚀 Iniciando o banco de testes COM OTIMIZAÇÃO de hiperparâmetros...\n")

# --- Loop principal modificado ---
for nome_modelo, config in modelos_a_testar.items():
    for nome_scaler, scaler in scalers_a_testar:
        for nome_encoder, encoder in encoders_a_testar:
            
            chave_pipeline = f"{nome_modelo}_{nome_scaler}_{nome_encoder}"
            print(f"--- Otimizando e Avaliando: {chave_pipeline} ---")

            # Construir o pré-processador dinamicamente (igual a antes)
            numeric_transformer = Pipeline(steps=[('scaler', scaler)])
            categorical_transformer = Pipeline(steps=[('encoder', encoder)])
            preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, colunas_numericas),
                    ('cat', categorical_transformer, colunas_categoricas)],
                remainder='passthrough')

            # Montar o pipeline final com o modelo base
            pipeline_final = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('classifier', config['model'])
            ])

            # ----------------------------------------------------
            # NOVA LÓGICA COM GridSearchCV
            # ----------------------------------------------------
            # 1. Configurar e rodar a busca
            param_grid = config['params']
            search = GridSearchCV(pipeline_final, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=0)
            search.fit(X, y)

            # 2. Salvar os melhores resultados da busca
            acuracia = search.best_score_
            melhores_parametros = search.best_params_
            melhor_pipeline = search.best_estimator_ # Este é o pipeline com os melhores parâmetros
            
            print(f"Melhor Acurácia (CV): {acuracia:.4f}")
            print(f"Melhores Parâmetros: {melhores_parametros}")

            # 3. Gerar predições e Matriz de Confusão com o MELHOR modelo encontrado
            y_pred = cross_val_predict(melhor_pipeline, X, y, cv=cv)
            cm = confusion_matrix(y, y_pred)
            
            # Armazenar os resultados
            resultados_finais.append({
                'combinacao': chave_pipeline,
                'acuracia_cv': acuracia,
                'melhores_parametros': melhores_parametros,
                'matriz_confusao': cm
            })

            # Plotar e salvar a matriz de confusão (igual a antes)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm)
            fig, ax = plt.subplots(figsize=(6, 5))
            disp.plot(ax=ax, cmap='Blues')
            ax.set_title(f'Matriz de Confusão (Otimizada)\n{chave_pipeline}')
            plt.tight_layout()
            plt.savefig(output_path / f'cm_{chave_pipeline}.png')
            plt.close(fig)
            print("-" * (len(chave_pipeline) + 25) + "\n")

print("✅ Banco de testes concluído!")

# Criar um DataFrame com o resumo dos resultados
df_resultados = pd.DataFrame(resultados_finais).drop(columns=['matriz_confusao'])
# df_resultados = df_resultados.sort_values(by='acuracia', ascending=False).reset_index(drop=True)

# Exibir a tabela de classificação
print("\n🏆 Tabela de Classificação Final das Combinações 🏆")
try:
    from IPython.display import display
    display(df_resultados)
except ImportError:
    print(df_resultados.to_string())

# Identificar a melhor combinação
melhor_combinacao = resultados_finais[df_resultados.index[0]]
print(f"\n🎉 Melhor combinação encontrada: '{melhor_combinacao['combinacao']}' com acurácia de {melhor_combinacao['acuracia']:.4f}")


🚀 Iniciando o banco de testes COM OTIMIZAÇÃO de hiperparâmetros...

--- Otimizando e Avaliando: LightGBM_StandardScaler_OneHotEncoder ---
[LightGBM] [Info] Number of positive: 10000, number of negative: 10000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006991 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2306
[LightGBM] [Info] Number of data points in the train set: 20000, number of used features: 817
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

c:\Users\PICHAU\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [6, 7, 13, 16, 20] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\PICHAU\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8000, number of negative: 8000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001923 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2139
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 739
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

c:\Users\PICHAU\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [6, 7, 13, 16, 18, 20, 22] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\PICHAU\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8000, number of negative: 8000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005582 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2145
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 742
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

...

c:\Users\PICHAU\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [6, 7, 13, 16, 20] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\PICHAU\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [6, 7, 13, 16, 18, 20, 22] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\PICHAU\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [6, 7, 13, 16] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\PICHAU\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [6, 7, 13] during tr

--------------------------------------------------------------------------

✅ Banco de testes concluído!

🏆 Tabela de Classificação Final das Combinações 🏆


c:\Users\PICHAU\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [6, 7, 13, 16, 18] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,combinacao,acuracia_cv,melhores_parametros
0,LightGBM_StandardScaler_OneHotEncoder,0.60100,"{'classifier__learning_rate': 0.1, 'classifier..."
1,LightGBM_MinMaxScaler_OneHotEncoder,0.59910,"{'classifier__learning_rate': 0.1, 'classifier..."
2,LightGBM_QuantileTransformer_OneHotEncoder,0.60050,"{'classifier__learning_rate': 0.1, 'classifier..."
3,LightGBM_PowerTransformer_OneHotEncoder,0.60135,"{'classifier__learning_rate': 0.1, 'classifier..."
4,CatBoost_StandardScaler_OneHotEncoder,0.59980,"{'classifier__depth': 6, 'classifier__iteratio..."
5,CatBoost_MinMaxScaler_OneHotEncoder,0.59980,"{'classifier__depth': 6, 'classifier__iteratio..."
6,CatBoost_QuantileTransformer_OneHotEncoder,0.60080,"{'classifier__depth': 4, 'classifier__iteratio..."
7,CatBoost_PowerTransformer_OneHotEncoder,0.60215,"{'classifier__depth': 6, 'classifier__iteratio..."
8,XGBoost_StandardScaler_OneHotEncoder,0.60225,"{'classifier__learning_rate': 0.1, 'classifier..."
9,XGBoost_MinMaxScaler_OneHotEncoder,0.60345,"{'classifier__learning_rate': 0.2, 'classifier..."


KeyError: 'acuracia'

In [42]:
bins = [17, 25, 40, 60, 110]
labels = ['Jovem', 'Adulto', 'MeiaIdade', 'Senior']
df_predicao['faixa_etaria'] = pd.cut(df_predicao['idade'], bins=bins, labels=labels, right=False)

colunas_categoricas.append('faixa_etaria')

preprocessor_vencedor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('scaler', StandardScaler())]), colunas_numericas),
        ('cat', Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))]), colunas_categoricas)
    ],
    remainder='passthrough'
)

X_train_predict = df_predicao

for col in colunas_categoricas:
    # Verifica se a coluna realmente existe no DataFrame antes de tentar converter
    if col in X_train_predict.columns:
        X_train_predict[col] = X_train_predict[col].astype(str)

# 3.3. Crie o pipeline final com o modelo vencedor
pipeline_vencedor = Pipeline(steps=[
    ('preprocessor', preprocessor_vencedor),
    ('classifier', LogisticRegression(random_state=42, max_iter=2000, C=0.1, solver='liblinear'))
])

# O melhor modelo, já treinado com todos os dados de treino, está em:
modelo_final_pronto = pipeline_vencedor.fit(X, y)

# ======================================================================
# 5. Predição no Conjunto de Teste Final
# ======================================================================
print("\nRealizando predições no conjunto de teste de 5.000 amostras...")
predicoes_finais = modelo_final_pronto.predict(X_train_predict)

# ======================================================================
# 6. Criação do Arquivo de Submissão
# ======================================================================
print("Criando o arquivo de submissão...")

# Criar um DataFrame para a submissão
df_submissao = pd.DataFrame({
    'id_solicitante': pd.read_csv('conjunto_de_teste.csv')['id_solicitante'], # Pega o ID do arquivo original
    'inadimplente': predicoes_finais
})

# Salvar o arquivo no formato .csv, sem o índice do pandas
nome_arquivo_submissao = f'submissao_final_.csv'
df_submissao.to_csv(nome_arquivo_submissao, index=False)

print(f"\n🎉 Arquivo '{nome_arquivo_submissao}' criado com sucesso e pronto para envio! 🎉")


Realizando predições no conjunto de teste de 5.000 amostras...
Criando o arquivo de submissão...

🎉 Arquivo 'submissao_final_.csv' criado com sucesso e pronto para envio! 🎉


c:\Users\PICHAU\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [6, 7, 13, 16, 18, 20] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
import shap

preprocessor_treinado = modelo_final_pronto.named_steps['preprocessor']
modelo_treinado = modelo_final_pronto.named_steps['classifier']

# ======================================================================
# PASSO 2: TRANSFORMAR OS DADOS E OBTER OS NOMES DAS FEATURES
# ======================================================================
# O SHAP precisa dos dados da mesma forma que o modelo os "vê", ou seja,
# após o pré-processamento (scaling, one-hot encoding, etc.).

print("Transformando os dados de treino para o formato que o modelo entende...")
print("Transformando os dados de treino para o formato que o modelo entende...")
X_transformado_sparse = preprocessor_treinado.transform(X)

# Obter os nomes das colunas após a transformação
try:
    nomes_features_finais = preprocessor_treinado.get_feature_names_out()
    nomes_features_finais = [str(col) for col in nomes_features_finais]
except Exception as e:
    print(f"Não foi possível obter os nomes das features automaticamente: {e}")
    nomes_features_finais = None

# A CORREÇÃO: Converter a matriz esparsa para um array denso (.toarray())
# antes de criar o DataFrame.
print("Convertendo matriz esparsa para densa...")
if nomes_features_finais:
    X_transformado_df = pd.DataFrame(X_transformado_sparse.toarray(), columns=nomes_features_finais)
else:
    X_transformado_df = pd.DataFrame(X_transformado_sparse.toarray())


# ======================================================================
# PASSO 3: CALCULAR OS VALORES SHAP
# ======================================================================
# Para modelos baseados em árvore como CatBoost, XGBoost e LightGBM,
# usamos o TreeExplainer, que é muito rápido e preciso.

print("Criando o Explainer e calculando os valores SHAP (isso pode levar um momento)...")
explainer = shap.TreeExplainer(modelo_treinado)
shap_values = explainer.shap_values(X_transformado_df)

print("Cálculo SHAP concluído!")

# ======================================================================
# PASSO 4: GERAR E INTERPRETAR O GRÁFICO
# ======================================================================
print("Gerando o gráfico de resumo SHAP...")

# --- Gráfico de Importância Global (similar ao feature_importances_) ---
plt.figure()
shap.summary_plot(shap_values, X_transformado_df, plot_type="bar", show=False)
plt.title("Importância Global das Features (SHAP)")
plt.tight_layout()
plt.savefig('shap_importancia_global.png')
plt.show()


# --- Gráfico de Resumo (O MAIS PODEROSO) ---
plt.figure()
shap.summary_plot(shap_values, X_transformado_df, show=False)
# O título é adicionado manualmente depois para melhor controle
plt.title("Resumo do Impacto das Features no Modelo (SHAP)")
plt.tight_layout()
plt.savefig('shap_summary_plot.png')
plt.show()